In [158]:
from graphreportr import GraphReportr
import numpy as np
import json
import glob
import networkx as nx


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.svm import SVR



import csv

In [126]:
def get_areas():
    areas = {}
    for f in glob.glob('./vtd-adjacency-graphs-master/vtd-adjacency-graphs/*/rook.json'):
        st = f[51:53]
        
        districts = set()
        with open(f) as h:
            data = json.load(h)
        g  = nx.readwrite.json_graph.adjacency_graph(data)
        try:
            for node in g.nodes(data=True):
                districts.add(node[1]['CD'])
                chk = node[1]['ALAND10']
                chk = node[1]['AWATER10']
        except: 
            continue
        for d in districts:

            ar = sum([float(n[1]['ALAND10'])+float(n[1]['AWATER10']) for n in g.nodes(data=True) if n[1]['CD'] == d])                
                
            areas[(st,d)]=ar

                
    return areas

In [153]:
a = get_areas()
full_dat = np.genfromtxt("rook_stats.csv",delimiter=',')
np.random.shuffle(full_dat)
new_dat = []
for row in full_dat[:,:]:
    if row.tolist()[0] !=36 and row.tolist()[0] !=41:
        new_dat.append(row)
        
        
full_dat = np.array(new_dat)

dat = full_dat[:,2:]
print(dat.shape)


ar_v = []
for e in full_dat[:,:2]:
    e = e.tolist()
    k = (str(int(e[0])),str(int(e[1])))
    if len(k[0]) == 1: k = ('0'+k[0],k[1])
    if len(k[1]) == 1: k = (k[0],'0'+k[1])
    ar_v.append(a[k])
ar_v = np.array(ar_v).reshape(-1,1)
print(ar_v.shape)

(344, 14)
(344, 1)


In [164]:
scaler = StandardScaler(copy=True)
norm_dat = scaler.fit_transform(dat)
norm_area = scaler.fit_transform(ar_v)



pca = PCA(n_components=4)
pca.fit(norm_dat)


Xtrain = dat[:-30,:]
Xtest = dat[-30:,:]

Ytrain = ar_v[:-30,:]
Ytest = ar_v[-30:,:]

km = KMeans(n_clusters=4).fit(norm_dat)

f = km.predict(norm_dat).tolist()

for j in range(km.n_clusters):
    for i in range(len(f)):

        if f[i]==j:
            d = full_dat[i,:2].tolist()
            #print("District {}-{} in class {}".format(str(int(d[0])),str(int(d[1])),f[i]))
            
            
with open('clust-4.csv','w') as outfile:
    writer = csv.writer(outfile)
    for j in range(km.n_clusters):
        for i in range(len(f)):

            if f[i]==j:
                d = full_dat[i,:2].tolist()
                d[0] = str(int(d[0]))
                d[1] = str(int(d[1]))
                if len(d[0]) == 1: d[0] = '0'+d[0]
                if len(d[1]) == 1: d[1] = '0'+d[1]

                writer.writerow([d[0]+d[1],f[i]])
                
                
svr = SVR()

svr.fit(Xtrain,Ytrain.ravel())
Ypred = svr.predict(Xtest)

svr.score(Xtest,Ytest)





spectral = SpectralClustering(
        n_clusters=4, eigen_solver='arpack',
        affinity="nearest_neighbors")
f = spectral.fit_predict(norm_dat).tolist()


for j in range(spectral.n_clusters):
    for i in range(len(f)):

        if f[i]==j:
            d = full_dat[i,:2].tolist()
            #print("District {}-{} in class {}".format(str(int(d[0])),str(int(d[1])),f[i]))
            
            
with open('spec_clust-4.csv','w') as outfile:
    writer = csv.writer(outfile)
    for j in range(spectral.n_clusters):
        for i in range(len(f)):

            if f[i]==j:
                d = full_dat[i,:2].tolist()
                d[0] = str(int(d[0]))
                d[1] = str(int(d[1]))
                if len(d[0]) == 1: d[0] = '0'+d[0]
                if len(d[1]) == 1: d[1] = '0'+d[1]

                writer.writerow([d[0]+d[1],f[i]])
                
with open('sptrees.csv','w') as outfile:
    writer = csv.writer(outfile)
    for j in range(spectral.n_clusters):
        for i in range(len(f)):

            if f[i]==j:
                d = full_dat[i,:2].tolist()
                t = full_dat[i,12]
                d[0] = str(int(d[0]))
                d[1] = str(int(d[1]))
                if len(d[0]) == 1: d[0] = '0'+d[0]
                if len(d[1]) == 1: d[1] = '0'+d[1]

                writer.writerow([d[0]+d[1],t])